In [12]:
from re import T
import pandas as pd
import os
import numpy as np
import math

TEAM_NAMES = {'Canada': 'Olympic (Women) - Canada', 
              'USA': 'Olympic (Women) - United States',
              'Finland': 'Olympic (Women) - Finland',
              'ROC': 'Olympic (Women) - Olympic Athletes from Russia',
              'Switzerland': 'Olympic (Women) - Switzerland'}


In [13]:
os.chdir('/mnt/c/Users/thepi/OneDrive/Documents/Python/Hockey/Big-Data-Cup-2022-Private')

In [14]:
game = '2022-02-08 Canada at USA'
pp = '2022-02-08 Canada at USA P1 PP1.csv'
location = "data/"+game+"/"+pp
MAX_FRAME_JUMP = 5


In [15]:
def get_names_date(game):
    spl = game.split(' ')
    date_comps = spl[0].split('-')
    date = '{}/{}/{}'.format(*[dt.strip('0') for dt in date_comps[::-1]])
    team_1 = TEAM_NAMES[spl[1]]
    team_2 = TEAM_NAMES[spl[3]]
    
    return team_1, team_2, date
    
def get_speed(track):
    assert (track['track_id'] == track['track_id'].iloc[0]).all()
    try:
        frame_diff = np.diff(track['frame_id'])
        nonvalid_speed = np.concatenate([[True],frame_diff>MAX_FRAME_JUMP])
        next_valid_speed = np.concatenate([frame_diff<=MAX_FRAME_JUMP,[False]])
        repl = nonvalid_speed & next_valid_speed
        idx = np.where(repl)[0]+1
        
        x_spd = np.concatenate([[np.nan],np.diff(track['x_ft'])/frame_diff])*30
        y_spd = np.concatenate([[np.nan],np.diff(track['y_ft'])/frame_diff])*30
        
        x_spd[repl] = x_spd[idx]
        y_spd[repl] = y_spd[idx]

    except Exception as e:
        print(np.diff(track['frame_id']))
        raise

    return(x_spd, y_spd)

In [40]:
def get_pp_tracking(power_play_info_file = 'pp_info.csv',
    play_by_play_data_file = 'pxp_womens_oly_2022_v2.csv',
    tracking_dir = 'data',
    pbp_dir = 'data'):

    play_by_play_data = pd.read_csv(os.path.join(tracking_dir,play_by_play_data_file))
    power_play_info = pd.read_csv(os.path.join(pbp_dir,power_play_info_file))

    #power_play_info['game_name'].loc[power_play_info['game_name']=='2022-02-14 USA at Finland'] = '2022-02-14 Finland at USA'
    power_play_info['relevant_events'] = np.nan

    relevant_eventses = []
    for ix, pp in power_play_info.iterrows():
        relevant_events = None
        game = pp['game_name']
        # print(game)
        #if game!='2022-02-14 Finland at USA':
        game_files = os.listdir(os.path.join(tracking_dir,game))
        tracking_data_name = os.path.join(tracking_dir,game,f'{game} P{pp["start_period"]} PP{pp["penalty_number"]}.csv')
        tracking_info_name = os.path.join(tracking_dir,game,f'videoShotsInfo_{game} P{pp["start_period"]} PP{pp["penalty_number"]}.csv')
        roster_info_name = os.path.join(tracking_dir, game, f'{game} roster.csv')
        roster_info = pd.read_csv(roster_info_name, index_col = 0)
        if os.path.split(tracking_data_name)[1] in game_files:
            print(tracking_data_name)
            tracking_data = pd.read_csv(tracking_data_name)
            #tracking_data['x_ft'] = tracking_data['x_ft'] - 7
            #tracking_data['y_ft'] = tracking_data['y_ft'] + 4

            tracking_data['vel_x'] = np.nan
            tracking_data['vel_y'] = np.nan
            tracking_data['angle'] = np.nan
            tr_ids = tracking_data.track_id.unique()
            for tr_id in tr_ids:
                idxs = tracking_data.track_id==tr_id
                track = tracking_data.loc[idxs]
                if len(track) <= 5:
                    continue
                x_spd,y_spd = get_speed(track)
                tracking_data.loc[idxs,'vel_x'] = x_spd
                tracking_data.loc[idxs,'vel_y'] = y_spd
                tracking_data['angle'] = np.arctan2(tracking_data['vel_y'],tracking_data['vel_x'])
            tracking_info = pd.read_csv(tracking_info_name)
        else:
            print('Tracking file missing:\n{}'.format(tracking_data_name))
            relevant_eventses.append(None)
            continue
        team_1, team_2, date = get_names_date(game)
        # print('{}\n{}\n{}\n'.format(team_1,team_2,date))
        relevant_events = play_by_play_data.loc[(play_by_play_data['game_date'] == date) & 
                                    ((play_by_play_data['team_name'] == team_1) |
                                        (play_by_play_data['team_name'] == team_2))]

        relevant_events = relevant_events.loc[((relevant_events['period']>= pp['start_period']) 
                                                & (relevant_events['clock_seconds'] <= pp['start_game_clock_seconds'])) &
                                              ((relevant_events['period']<= pp['end_period']) & 
                                              (relevant_events['clock_seconds'] >= pp['end_game_clock_seconds']))  ]


        relevant_events.index = np.arange(0, len(relevant_events.index), dtype = 'int')

        pl1_jn = roster_info.loc[relevant_events['player_name'],'jn']
        pl1_jn.index = relevant_events.index

        pl2_exists = list(relevant_events['player_name_2'].notna())
        pl2_jn = roster_info.loc[relevant_events['player_name_2'][pl2_exists],'jn']

        pl2_jn.index = relevant_events.index[pl2_exists]
        relevant_events['Player_1_num'] = pl1_jn
        relevant_events['Player_2_num'] = pl2_jn
        # print(relevant_events['frame_id_1'])

        # relevant_events =  relevant_events.loc[relevant_events['frame_id_1'].notna()]
        # print(relevant_events['frame_id_1'])
        # break
        tracks = []
        ### Continue from here - need to check if there is a next event

        for rw, event in relevant_events.iterrows():
            fr_id_1 = event['frame_id_1']
            if pd.isna(fr_id_1):
                tracks.append(None)
                continue
            fr_id_2 = event['frame_id_2']
            if pd.notna(fr_id_2):
                tracks.append(tracking_data.loc[(tracking_data['frame_id']>=fr_id_1) &
                                                     (tracking_data['frame_id']<=fr_id_2)])
            else:
                tracks.append(tracking_data.loc[tracking_data['frame_id']==fr_id_1])
            

        # for rw, event in relevant_events.iterrows():
        #     rel_shot = 0
        #     time_bonus = 0 if event['period'] == pp['start_period'] else 1200
        #     event_frame_time = (pp['start_game_clock_seconds'] - event['clock_seconds'] + time_bonus)*30.0 - 15

        #     if tracking_data_name == 'data/2022-02-16 Switzerland at Finland/2022-02-16 Switzerland at Finland P3 PP8.csv':
        #         print(pp['start_game_clock_seconds'])
        #         print(event['clock_seconds'])
        #     # print(len(relevant_events), rw)
        #     if rw < (len(relevant_events)-1):
        #         time_bonus = 0 if relevant_events.loc[rw+1,'period'] == pp['start_period'] else 1200
        #         next_event_frame_time = (pp['start_game_clock_seconds'] - relevant_events.loc[rw+1,'clock_seconds'] + time_bonus)*30.0 + 15
        #     else:
        #         next_event_frame_time = 10000000
        #     # print(event_frame_time, next_event_frame_time)
        #     # assert False
        #     while rel_shot<len(tracking_info.index):
        #         # print(tracking_info)
        #         if event_frame_time >= tracking_info.loc[rel_shot,'frame_start_shot']:
        #             if event_frame_time >= tracking_info.loc[rel_shot,'frame_end_shot']:
        #                 rel_shot+=1
        #                 if rel_shot==len(tracking_info.index):
        #                     tracks.append(None)
        #             else:
        #                 tracks.append(tracking_data.loc[(tracking_data['frame_id']>=(event_frame_time)) &
        #                                              (tracking_data['frame_id']<(next_event_frame_time))])
        #                 break

        #         elif next_event_frame_time > tracking_info.loc[rel_shot,'frame_start_shot']:
        #             tracks.append(tracking_data.loc[tracking_data['frame_id']<(next_event_frame_time)])
        #             break
        #         else:
        #             tracks.append(None)
        #             break
        relevant_events['tracks'] = tracks

        relevant_eventses.append(relevant_events)
    power_play_info['relevant_events'] = relevant_eventses
    return power_play_info


In [41]:
relevant_events = get_pp_tracking()
relevant_events.head()

data/2022-02-08 Canada at USA/2022-02-08 Canada at USA P1 PP1.csv
data/2022-02-08 Canada at USA/2022-02-08 Canada at USA P1 PP2.csv
data/2022-02-08 Canada at USA/2022-02-08 Canada at USA P2 PP3.csv
Tracking file missing:
data/2022-02-08 Canada at USA/2022-02-08 Canada at USA P2 PP4.csv
data/2022-02-08 Canada at USA/2022-02-08 Canada at USA P2 PP5.csv
data/2022-02-08 Canada at USA/2022-02-08 Canada at USA P3 PP6.csv
data/2022-02-08 Canada at USA/2022-02-08 Canada at USA P3 PP7.csv
data/2022-02-08 ROC at Finland/2022-02-08 ROC at Finland P1 PP1.csv
data/2022-02-08 ROC at Finland/2022-02-08 ROC at Finland P2 PP2.csv
Tracking file missing:
data/2022-02-08 ROC at Finland/2022-02-08 ROC at Finland P2 PP3.csv
Tracking file missing:
data/2022-02-08 ROC at Finland/2022-02-08 ROC at Finland P2 PP4.csv
data/2022-02-08 ROC at Finland/2022-02-08 ROC at Finland P2 PP5.csv
data/2022-02-08 ROC at Finland/2022-02-08 ROC at Finland P3 PP6.csv
data/2022-02-12 Switzerland at ROC/2022-02-12 Switzerland at 

,game_name,penalty_number,start_video_clock_seconds,end_video_clock_seconds,start_period,end_period,start_game_clock_seconds,end_game_clock_seconds,relevant_events
0,2022-02-08 Canada at USA,1,1278,1314,1,1,386,350,game_date season_year team...
1,2022-02-08 Canada at USA,2,1624,1744,1,1,216,96,game_date season_year ...
2,2022-02-08 Canada at USA,3,409,562,2,2,991,871,game_date season_year ...
3,2022-02-08 Canada at USA,4,898,898,2,2,613,613,None
4,2022-02-08 Canada at USA,5,1690,1927,2,2,272,152,game_date season_year ...


In [42]:
relevant_events.loc[0,'relevant_events'].head()

,game_date,season_year,team_name,opp_team_name,venue,period,clock_seconds,situation_type,goals_for,goals_against,...,x_coord_2,y_coord_2,event_detail_1,event_detail_2,event_detail_3,frame_id_1,frame_id_2,Player_1_num,Player_2_num,tracks
6,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,375,5 on 4,0,0,...,139.0,5.0,NaN,NaN,NaN,344.0,370.0,19,29.0,frame_id period track_id team_id team_n...
7,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,373,5 on 4,0,0,...,143.0,19.0,NaN,NaN,NaN,420.0,443.0,29,6.0,frame_id period track_id team_id team_...
8,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,366,5 on 4,0,0,...,130.0,17.0,NaN,NaN,NaN,605.0,639.0,6,29.0,frame_id period track_id team_id team_...
9,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,365,5 on 4,0,0,...,133.0,55.0,NaN,NaN,NaN,650.0,670.0,29,23.0,frame_id period track_id team_id team_...
10,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,363,5 on 4,0,0,...,176.0,42.0,NaN,NaN,NaN,701.0,718.0,23,19.0,frame_id period track_id team_id team_...


In [30]:
relevant_events.loc[0,'relevant_events'].tracks.iloc[6]

,frame_id,period,track_id,team_id,team_name,jersey_number,x_ft,y_ft,video_shot,game_seconds,vel_x,vel_y,angle
203,330,1,8,Dark,USA,25,158.030516,50.052628,1,376,-7.941585,-0.436859,-3.086639
204,330,1,7,Dark,USA,5,189.192611,58.370179,1,376,0.550749,10.684505,1.519295
205,330,1,5,Dark,USA,28,167.728670,40.551812,1,376,0.488952,1.950640,1.325194
206,330,1,4,Light,Canada,19,195.274569,67.042075,1,376,-8.919522,4.312770,2.691216
207,330,1,3,Dark,USA,35,186.791291,43.594256,1,376,-2.475153,-0.212571,-3.055921
...,...,...,...,...,...,...,...,...,...,...,...,...,...
821,404,1,4,Light,Canada,19,182.150170,61.593181,1,373,0.244986,-11.055250,-1.548640
822,404,1,7,Dark,USA,5,173.488274,59.395426,1,373,-1.012683,3.871421,1.826643
823,404,1,2,Dark,USA,2,175.493738,44.706158,1,373,1.527054,-6.361436,-1.335205
824,404,1,5,Dark,USA,28,161.507374,47.719855,1,373,0.551709,5.255918,1.466210


In [9]:
relevant_events.to_json(r'data/BDC_2022_all_data.json')

In [24]:
relevant_events2 = relevant_events.loc[~relevant_events['relevant_events'].isna()]
the_truth = (relevant_events2.loc[0,'relevant_events']['event_type'] == 'Direct')&(~relevant_events2.loc[0,'relevant_events']['tracks'].isna())
r0_direct = relevant_events2.loc[0,'relevant_events'].loc[the_truth]

for r in relevant_events2.index[1:]:
    r_the_truth = (relevant_events2.loc[r,'relevant_events']['event_type'] == 'Direct')&(~relevant_events2.loc[r,'relevant_events']['tracks'].isna())
    r_new_direct = relevant_events2.loc[r,'relevant_events'].loc[r_the_truth]
    r0_direct=pd.concat([r0_direct, r_new_direct])
r0_direct = r0_direct.reset_index(drop=True)

In [ ]:
r0_direct.to_json(r'data/BDC_2022_all_direct_wFrames.json')

In [26]:
r0_direct.iloc[383]
# r0_direct.loc[383,'tracks']

game_date                                                   16/2/2022
season_year                                                      2021
team_name                                   Olympic (Women) - Finland
opp_team_name                           Olympic (Women) - Switzerland
venue                                                            home
period                                                              3
clock_seconds                                                      68
situation_type                                                 5 on 4
goals_for                                                           3
goals_against                                                       0
player_name                                               Noora Tulus
event                                                            Play
event_successful                                                    t
x_coord                                                           176
y_coord             

In [19]:
truth2 = r0_direct.loc[0,'tracks']['frame_id']==r0_direct.frame_id_1[0]
r_frame_direct = r0_direct.loc[0,'tracks'].loc[truth2]
r_direct_oneFrame = []

for r in r0_direct.index:
    r_truth2 = r0_direct.loc[r,'tracks']['frame_id']==r0_direct.frame_id_1[r]
    r2_frame_direct = r0_direct.loc[r,'tracks'].loc[r_truth2]
    r_direct_oneFrame.append(r2_frame_direct)

r0_direct['tracks']=r_direct_oneFrame

In [23]:
r0_direct.loc[-1,'tracks']

KeyError: -1

In [20]:
[np.where([len(ln)==0 for ln in r0_direct['tracks']])]

[(array([ 25,  29,  31,  49,  50,  51,  52,  53,  54,  55,  56,  59,  64,
          76,  92,  95, 142, 162, 163, 164, 165, 166, 175, 176, 177, 178,
         179, 180, 181, 182, 183, 184, 185, 186, 207, 208, 209, 210, 211,
         212, 213, 214, 215, 219, 221, 222, 225, 226, 239, 246, 247, 259,
         262, 265, 278, 279, 301, 302, 303, 318, 319, 320, 321, 332, 333,
         334, 335, 337, 338, 339, 340, 341, 348, 350, 357, 359, 360, 361,
         364, 365, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386]),)]

In [41]:
r0_direct.drop([ 25,  29,  31,  49,  50,  51,  52,  53,  54,  55,  56,  59,  64,
          76,  92,  95, 142, 162, 163, 164, 165, 166, 175, 176, 177, 178,
         179, 180, 181, 182, 183, 184, 185, 186, 207, 208, 209, 210, 211,
         212, 213, 214, 215, 219, 221, 222, 225, 226, 239, 246, 247, 259,
         262, 265, 278, 279, 301, 302, 303, 318, 319, 320, 321, 332, 333,
         334, 335, 337, 338, 339, 340, 341, 348, 350, 357, 359, 360, 361,
         364, 365, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386], axis=0, inplace=True)

In [43]:
r0_direct
r0_direct = r0_direct.reset_index(drop=True)

In [ ]:
r0_direct.to_json(r'data/BDC_2022_passOnly.json')